In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from __future__ import annotations
from typing import Protocol, Iterator, Tuple, TypeVar, Optional

In [88]:
from termcolor import colored

In [170]:
import numpy as np

In [3]:
T = TypeVar('T')
Location = TypeVar('Location')

In [4]:
class Graph(Protocol):
    def neighbors(self, id: Location) -> list[Location]: pass
    
class SimpleGraph:
    def __init__(self):
        self.edges: dict[Location,list[Location]]={}
    
    def neighbors(self, id: Location) -> list[Location]:
        return self.edges[id]


In [5]:
example_graph = SimpleGraph()
example_graph.edges = {
    'A': ['B'],
    'B': ['C'],
    'C': ['B', 'D', 'F'],
    'D': ['C', 'E'],
    'E': ['F'],
    'F': [],
}

In [6]:
import collections

In [7]:
class Queue:
    def __init__(self):
        self.elements = collections.deque()
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, x: T):
        self.elements.append(x)
        
    def get(self) -> T:
        return self.elements.popleft()

In [147]:
# utility functions for dealing with square grids
# id를 width로 나눈 (나머지, 몫)을 돌려준다.
# 이는 (x,y) 좌표에 해당
def from_id_with(id, width):
    return (id % width, id // width)

def draw_tile(graph, id, style):
    r = " . "
    if 'number' in style and id in style['number']:
        r = " %-2d" % style['number'][id]
    if 'point_to' in style and style['point_to'].get(id, None) is not None:
        (x1, y1) = id
        (x2, y2) = style['point_to'][id]
        if x2 == x1 + 1: r = " > "
        if x2 == x1 - 1: r = " < "
        if y2 == y1 + 1: r = " v "
        if y2 == y1 - 1: r = " ^ "
    if 'path' in style and id in style['path']:   r = colored(" @ ","green","on_grey")
    if 'start' in style and id == style['start']: r = colored(" A ","red","on_yellow")
    if 'goal' in style and id == style['goal']:   r = colored(" Z ","red","on_yellow")
    if id in graph.walls: r = "###"
    return r

def draw_grid(graph, **style):
    print("___" * graph.width)
    for y in range(graph.height):
        for x in range(graph.width):
            print("%s" % draw_tile(graph,(x,y),style),end="")
        print()
    print("~~~" * graph.width)

In [141]:
GridLocation = Tuple[int, int]

In [178]:
class SquareGrid:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.walls: list[GridLocation] = []
    
    def in_bounds(self, id: GridLocation) -> bool:
        (x,y) = id
        return 0 <= x < self.width and 0 <= y < self.height

    def passible(self, id: GridLocation) -> bool:
        return id not in self.walls
    
    def neighbors(self, id:GridLocation) -> Iterator[GridLocation]:
        (x,y) = id
        neighbors = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)] # E W N S
        # see "Ugly paths" section for an explanation:
#         if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        # grid 범위에 있는 이웃 grid들만 필터링
        results = filter(self.in_bounds, neighbors)
        # wall등 장에물에 있는 grid는 제외
        results = filter(self.passible, results)
        return results

In [209]:
class SquareGridNeighborOrder(SquareGrid):
    def neighbors(self, id:GridLocation) -> list:
        (x,y) = id
        neighbors = [(x + dx, y + dx) for (dx, dy) in self.NEIGHBOR_ORDER]
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passible, results)
        return list(results)

In [182]:
class WeightedGraph(Graph):
    def cost(self, from_id: Location, to_id: Location)  -> float: pass
    
class GridWithWeights(SquareGrid):
    def __init__(self, width: int, height: int):
        super().__init__(width, height)
        self.weights: dict[GridLocation, float]= {}
    
    def cost(self, from_node: GridLocation, to_node: GridLocation) -> float:
        # 값이 없어면 1
        return self.weights.get(to_node, 1)

In [211]:
class GridWithAdjustedWeights(GridWithWeights):
    def cost(self, from_node, to_node):
        prev_cost = super().cost(from_node, to_node)
        nudge = 0
        (x1, y1) = from_node
        (x2, y2) = to_node
        if(x1 + y1) % 2 == 0 and x2 != x1: nudge = 1
        if(x1 + y1) % 2 == 1 and y2 != y1: nudge = 1
        return prev_cost + 0.001 * nudge

In [184]:
import heapq

In [185]:
class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, T]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: T, priority: float):
        heapq.heappush(self.elements,(priority, item))
        
    def get(self) -> T:
        return heapq.heappop(self.elements)[1]

### 경로재구성

In [186]:
def reconstruct_path(came_from: dict[Location, Location],
                    start: Location, goal: Location) -> list[Location]:
    current: Location = goal
    path: list[Location] = []
    if goal not in came_from:
        return []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()
    return path

### Breath First Search

In [187]:
def breadth_first_search(graph: Graph, start: Location, goal: Location, print_log: bool=False):
    frontier = Queue()
    frontier.put(start)
    came_from: dict[Location, Optional[Location]] = {}
    came_from[start] = None
    
    while not frontier.empty():
        current: Location = frontier.get()
        if print_log : print("  Visiting %s" % current)
        
        if current == goal:
            if print_log: print( " Goal! %s" % current)
            break
            
        for next in graph.neighbors(current):
            if next not in came_from:
                frontier.put(next)
                came_from[next] = current
    return came_from

### Dijkstra Search

In [188]:
def dijkstra_search(graph: WeightedGraph, start: Location, goal: Location):
    frontier = PriorityQueue()
    frontier.put(start,0)
    came_from: dict[Location, Optional[Location]] = {}
    cost_so_far: dict[Location, float] = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current: Location = frontier.get()
        
        if current == goal:
            break
        
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost
                frontier.put(next, priority)
                came_from[next] = current
    
    return came_from, cost_so_far
    

### A* Search

In [189]:
# Manhattan distance
def heuristic(a: GridLocation, b: GridLocation) -> float:
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

In [190]:
def a_start_search(graph: WeightedGraph, start: Location, goal: Location):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from: dict[Location, Optional[Location]] = {}
    cost_so_far: dict[Location, float] = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current: Location = frontier.get()
            
        if current == goal:
            break
            
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, goal)
                frontier.put(next, priority)
                came_from[next] = current
    return came_from, cost_so_far

### Example : Simple Graph

In [191]:
example_graph = SimpleGraph()
example_graph.edges = {
    'A': ['B'],
    'B': ['C'],
    'C': ['B', 'D', 'F'],
    'D': ['C', 'E'],
    'E': ['F'],
    'F': [],
}

In [192]:
print('Reachable from A:')
breadth_first_search(example_graph, 'A', 'F')

Reachable from A:


{'A': None, 'B': 'A', 'C': 'B', 'D': 'C', 'F': 'C', 'E': 'D'}

In [193]:
print('Reachable from E:')
breadth_first_search(example_graph, 'D', 'B', print_log=True)

Reachable from E:
  Visiting D
  Visiting C
  Visiting E
  Visiting B
 Goal! B


{'D': None, 'C': 'D', 'E': 'D', 'B': 'C', 'F': 'C'}

### Example : Grid

#### 입력데이터 : DIAGRAM1_WALLS

In [194]:
# data frin main article
DIAGRAM1_WALLS = [from_id_with(id, width=30) for id in [
    21,22,51,52,81,82,93,94,111,112,123,124,133,134,141,142,
    153,154,163,164,171,172,173,174,175,183,184,193,194,201,
    202,203,204,205,213,214,223,224,243,244,253,254,273,274,
    283,284,303,304,313,314,333,334,343,344,373,374,403,404,
    433,434]]

#### Square grid display

In [195]:
g = SquareGrid(30,15)
g.walls = DIAGRAM1_WALLS
draw_grid(g)

__________________________________________________________________________________________
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ############### .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ############### .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

### Example : Breadth Search

In [196]:
g = SquareGrid(30,15)
g.walls = DIAGRAM1_WALLS
start =(8,7)
goal = None
parents = breadth_first_search(g,start,goal)
draw_grid(g, point_to=parents,start=start, goal=goal)

__________________________________________________________________________________________
 v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v ###### v  v  v  v  v  v  v 
 v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v ###### v  v  v  v  v  v  v 
 >  >  >  >  >  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v ###### v  v  v  v  v  v  v 
 ^  ^  ^ ###### v  v  v  v  v  v  v  v  <  <  <  <  <  <  <  < ###### v  v  v  v  v  v  v 
 ^  ^  ^ ###### v  v  v  v  v  v  v  v ###### ^  ^  ^  ^  ^  ^ ###### >  >  >  v  v  v  v 
 ^  ^  ^ ###### v  v  v  v  v  v  v  v ###### ^  ^  ^  ^  ^  ^ ############### v  v  v  v 
 ^  ^  ^ ###### v  v  v  v  v  v  v  v ###### ^  ^  ^  ^  ^  ^ ############### v  v  v  v 
 ^  ^  ^ ###### >  >  >  A  <  <  <  < ###### ^  ^  ^  ^  ^  ^  <  <  <  <  <  <  <  <  < 
 v  v  v ###### ^  ^  ^  ^  ^  ^  ^  ^ ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^ 
 v  v  v ###### ^  ^  ^  ^  ^  ^  ^  ^ ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^ 

In [219]:
g = SquareGrid(30, 15)
g.walls = DIAGRAM1_WALLS

start = (1, 9)
goal = (23, 2)
came_from = breadth_first_search(g, start, goal)
draw_grid(g, path=reconstruct_path(came_from, start=start, goal=goal),
         point_to=came_from, start=start, goal= goal)

__________________________________________________________________________________________
 v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v ###### .  .  v  v  v  v  . 
 v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v ###### .  v  v  v  v  v  v 
 v  v  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @ ###### Z  v  v  v  v  v  v 
 v  v  @ ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  @ ###### @  v  v  v  v  v  v 
 v  v  @ ###### ^  ^  ^  ^  ^  ^  ^  ^ ###### ^  ^  ^  ^  ^  @ ###### @  @  @  @  v  v  v 
 v  v  @ ###### ^  ^  ^  ^  ^  ^  ^  ^ ###### ^  ^  ^  ^  ^  @ ############### @  v  v  v 
 v  v  @ ###### v  v  v  v  v  v  v  v ###### ^  ^  ^  ^  ^  @ ############### @  v  v  v 
 v  v  @ ###### v  v  v  v  v  v  v  v ###### ^  ^  ^  ^  ^  @  @  @  @  @  @  @  <  <  < 
 v  v  @ ###### v  v  v  v  v  v  v  v ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^ 
 >  A  @ ###### v  v  v  v  v  v  v  v ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^ 

### Example : Dijkstra Search - constant weight : cost = 1

In [217]:
g = GridWithWeights(30, 15)
g.walls = DIAGRAM1_WALLS

start = (1, 9)
goal = (23, 2)
draw_grid(g, path=reconstruct_path(came_from, start=start, goal=goal),
         point_to=came_from, start=start, goal=goal)

__________________________________________________________________________________________
 v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v ###### .  .  .  .  .  .  . 
 v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v ###### .  .  .  .  .  .  . 
 >  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @ ###### Z  .  .  .  .  .  . 
 >  @  < ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  @ ###### @  v  v  v  .  .  . 
 >  @  < ###### ^  ^  ^  ^  ^  ^  ^  ^ ###### ^  ^  ^  ^  ^  @ ###### @  @  @  @  <  .  . 
 >  @  < ###### ^  ^  ^  ^  ^  ^  ^  ^ ###### ^  ^  ^  ^  ^  @ ############### @  <  .  . 
 >  @  < ###### v  <  <  <  <  <  <  < ###### ^  ^  ^  ^  ^  @ ############### @  <  .  . 
 >  @  < ###### v  <  <  <  <  <  <  < ###### ^  ^  ^  ^  ^  @  @  @  @  @  @  @  <  .  . 
 >  @  < ###### v  <  <  <  <  <  <  < ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  <  <  <  .  .  . 
 >  A  < ###### v  <  <  <  <  <  <  < ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  <  <  .  .  .  . 

### Example : Dijkstra Search - weighted : cost = 좌표에 따라 다름

#### 입력 데이터 :diagram4

In [201]:
diagram4 = GridWithWeights(10,10)
diagram4.walls = [(1, 7), (1, 8), (2, 7), (2, 8), (3, 7), (3, 8)]
diagram4.weights = {loc: 5 for loc in [(3, 4), (3, 5), (4, 1), (4, 2),
                                       (4, 3), (4, 4), (4, 5), (4, 6),
                                       (4, 7), (4, 8), (5, 1), (5, 2),
                                       (5, 3), (5, 4), (5, 5), (5, 6),
                                       (5, 7), (5, 8), (6, 2), (6, 3),
                                       (6, 4), (6, 5), (6, 6), (6, 7),
                                       (7, 3), (7, 4), (7, 5)]}

In [202]:
start, goal = (1,4),(8,3)
came_from, cost_so_far = dijkstra_search(diagram4, start, goal)
draw_grid(diagram4, point_to = came_from, start=start, goal=goal)
print()
draw_grid(diagram4, path=reconstruct_path(came_from, start=start, goal=goal))

______________________________
 v  v  <  <  <  <  <  <  <  < 
 v  v  <  <  <  ^  ^  <  <  < 
 v  v  <  <  <  <  ^  ^  <  < 
 v  v  <  <  <  <  <  ^  Z  . 
 >  A  <  <  <  <  .  .  .  . 
 ^  ^  <  <  <  <  .  .  .  . 
 ^  ^  <  <  <  <  <  .  .  . 
 ^ ######### ^  <  v  v  .  . 
 ^ ######### v  v  v  <  <  . 
 ^  <  <  <  <  <  <  <  <  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

______________________________
 .  @  @  @  @  @  @  .  .  . 
 .  @  .  .  .  .  @  @  .  . 
 .  @  .  .  .  .  .  @  @  . 
 .  @  .  .  .  .  .  .  @  . 
 .  @  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


### Example : Dijkstra Search - 경로 없음 (Goal을 찾지 못한 경우)

In [203]:
diagram_nopath = GridWithWeights(10, 10)
diagram_nopath.walls = [(5, row) for row in range(10)]

start, goal = (1, 4), (8, 3)
came_from, cost_so_far = dijkstra_search(diagram_nopath, start, goal)
draw_grid(diagram_nopath, point_to=came_from, start=start, goal=goal)

______________________________
 v  v  <  <  < ### .  .  .  . 
 v  v  <  <  < ### .  .  .  . 
 v  v  <  <  < ### .  .  .  . 
 v  v  <  <  < ### .  .  Z  . 
 >  A  <  <  < ### .  .  .  . 
 ^  ^  <  <  < ### .  .  .  . 
 ^  ^  <  <  < ### .  .  .  . 
 ^  ^  <  <  < ### .  .  .  . 
 ^  ^  <  <  < ### .  .  .  . 
 ^  ^  <  <  < ### .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


### Example : Dijkstra - Distance fields

In [204]:
start, goal = (1, 4), None
came_from, cost_so_far = dijkstra_search(diagram4, start, goal)
draw_grid(diagram4, number=cost_so_far, start=start)

______________________________
 5  4  5  6  7  8  9  10 11 12
 4  3  4  5  10 13 10 11 12 13
 3  2  3  4  9  14 15 12 13 14
 2  1  2  3  8  13 18 17 14 15
 1  A  1  6  11 16 21 20 15 16
 2  1  2  7  12 17 22 21 16 17
 3  2  3  4  9  14 19 16 17 18
 4 ######### 14 19 18 15 16 17
 5 ######### 15 16 13 14 15 16
 6  7  8  9  10 11 12 13 14 15
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


### Example: A* Search

In [205]:
start, goal = (1,4), (8,3)
came_from, cost_so_far = a_start_search(diagram4, start, goal)
draw_grid(diagram4, number=cost_so_far, start=start)

______________________________
 5  4  5  6  7  8  9  10 11 12
 4  3  4  5  10 13 10 11 12 13
 3  2  3  4  9  14 15 12 13 14
 2  1  2  3  8  13 .  17 14 . 
 1  A  1  6  11 .  .  .  .  . 
 2  1  2  7  12 .  .  .  .  . 
 3  2  3  4  9  .  .  .  .  . 
 4 ######### .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [206]:
draw_grid(diagram4, point_to = came_from, start=start, goal=goal)
print()
draw_grid(diagram4, path =  reconstruct_path(came_from, start=start, goal=goal))

______________________________
 v  v  v  v  <  <  <  <  <  < 
 v  v  v  v  <  ^  ^  <  <  < 
 v  v  v  v  <  <  ^  ^  <  < 
 >  v  <  <  <  <  .  ^  Z  . 
 >  A  <  <  <  .  .  .  .  . 
 ^  ^  ^  <  <  .  .  .  .  . 
 ^  ^  ^  <  <  .  .  .  .  . 
 ^ ######### .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

______________________________
 .  .  .  @  @  @  @  .  .  . 
 .  .  .  @  .  .  @  @  .  . 
 .  .  .  @  .  .  .  @  @  . 
 .  @  @  @  .  .  .  .  @  . 
 .  @  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [215]:
start = (1, 9)
goal = (23, 2)
came_from, cost_so_far = a_start_search(g, start, goal)
draw_grid(g, path=reconstruct_path(came_from, start=start, goal=goal),
         point_to=came_from, start=start, goal=goal)

__________________________________________________________________________________________
 v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v ###### .  .  .  .  .  .  . 
 v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v ###### .  .  .  .  .  .  . 
 >  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @  @ ###### Z  .  .  .  .  .  . 
 >  @  < ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  @ ###### @  v  v  v  .  .  . 
 >  @  < ###### ^  ^  ^  ^  ^  ^  ^  ^ ###### ^  ^  ^  ^  ^  @ ###### @  @  @  @  <  .  . 
 >  @  < ###### ^  ^  ^  ^  ^  ^  ^  ^ ###### ^  ^  ^  ^  ^  @ ############### @  <  .  . 
 >  @  < ###### v  <  <  <  <  <  <  < ###### ^  ^  ^  ^  ^  @ ############### @  <  .  . 
 >  @  < ###### v  <  <  <  <  <  <  < ###### ^  ^  ^  ^  ^  @  @  @  @  @  @  @  <  .  . 
 >  @  < ###### v  <  <  <  <  <  <  < ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  <  <  <  .  .  . 
 >  A  < ###### v  <  <  <  <  <  <  < ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  <  <  .  .  .  . 

In [208]:
draw_grid(g, number=cost_so_far, start=start, goal=goal)

__________________________________________________________________________________________
 10 9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28###### .  .  .  .  .  .  . 
 9  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27###### .  .  .  .  .  .  . 
 8  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26###### Z  .  .  .  .  .  . 
 7  6  7 ###### 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27###### 44 43 42 41 .  .  . 
 6  5  6 ###### 13 14 15 16 17 18 19 20###### 23 24 25 26 27 28###### 43 42 41 40 41 .  . 
 5  4  5 ###### 14 15 16 17 18 19 20 21###### 24 25 26 27 28 29############### 39 40 .  . 
 4  3  4 ###### 13 14 15 16 17 18 19 20###### 25 26 27 28 29 30############### 38 39 .  . 
 3  2  3 ###### 12 13 14 15 16 17 18 19###### 26 27 28 29 30 31 32 33 34 35 36 37 38 .  . 
 2  1  2 ###### 11 12 13 14 15 16 17 18###### 27 28 29 30 31 32 33 34 35 36 37 38 .  .  . 
 1  A  1 ###### 10 11 12 13 14 15 16 17###### 28 29 30 31 32 33 34 35 36 37 38 .  .  .  . 

#### Example : Test with custom order

In [221]:
def test_with_custom_order(neighbor_order):
    if neighbor_order:
        g = SquareGridNeighborOrder(30, 15)
        g.NEIGHBOR_ORDER = neighbor_order
    else:
        g = SquareGrid(30,15)
        
    g.walls = DIAGRAM1_WALLS
    start, goal = (8,7),(27,2)
    came_from = breadth_first_search(g, start, goal)
    draw_grid(g, path=reconstruct_path(came_from, start=start, goal=goal),
             point_to = came_from, start=start, goal=goal)

In [222]:
test_with_custom_order([])

__________________________________________________________________________________________
 v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v ###### .  .  .  .  .  .  . 
 v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v ###### .  .  .  v  .  .  . 
 >  >  >  >  >  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v  v ###### .  .  v  v  Z  v  . 
 ^  ^  ^ ###### v  v  v  v  v  v  v  @  @  @  @  @  @  @  @  @ ###### .  v  v  v  @  v  v 
 ^  ^  ^ ###### v  v  v  v  v  v  v  @ ###### ^  ^  ^  ^  ^  @ ###### >  >  >  v  @  v  v 
 ^  ^  ^ ###### v  v  v  v  v  v  v  @ ###### ^  ^  ^  ^  ^  @ ############### v  @  v  v 
 ^  ^  ^ ###### v  v  v  v  v  v  v  @ ###### ^  ^  ^  ^  ^  @ ############### v  @  v  v 
 ^  ^  ^ ###### >  >  >  A  @  @  @  @ ###### ^  ^  ^  ^  ^  @  @  @  @  @  @  @  @  <  < 
 v  v  v ###### ^  ^  ^  ^  ^  ^  ^  ^ ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^ 
 v  v  v ###### ^  ^  ^  ^  ^  ^  ^  ^ ###### ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^ 

In [224]:
# 이것은 에러인데 다시 살펴 볼것
test_with_custom_order([(+1,0),(0,-1),(-1,0),(0,+1)])

__________________________________________________________________________________________
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  Z  .  . 
 .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  . ###### v  .  .  .  .  .  .  . ###### .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  . ###### .  v  .  .  .  .  .  . ###### .  .  .  .  .  . ############### .  .  .  . 
 .  .  . ###### .  .  v  .  .  .  .  . ###### .  .  .  .  .  . ############### .  .  .  . 
 .  .  . ###### .  .  .  A  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  ^  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  ^  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 